In [131]:
!pip install pyspark
!pip install datetime

In [180]:
from pyspark.sql import SparkSession
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json
import pandas as pd
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [133]:
df = pd.read_excel("/content/Intermediate_Task.xlsx")

In [134]:
print(df.dtypes)

Permit Number                              object
Permit Type                                 int64
Permit Type Definition                     object
Permit Creation Date                       object
Block                                      object
Lot                                        object
Street Number                               int64
Street Number Suffix                       object
Street Name                                object
Street Suffix                              object
Unit                                      float64
Unit Suffix                                object
Description                                object
Current Status                             object
Current Status Date                        object
Filed Date                                 object
Issued Date                                object
Completed Date                             object
First Construction Document Date           object
Structural Notification                    object


In [135]:
df.shape

(15364, 43)

In [136]:
df.head(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,2.01505E+11,4,sign - erect,05/06/2015,326,23,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1.380610e+12
1,2.01604E+11,4,sign - erect,04/19/2016,306,7,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1.420160e+12
2,2.01605E+11,3,additions alterations or repairs,05/27/2016,595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1.424860e+12
3,2.01611E+11,8,otc alterations permit,11/07/2016,156,11,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1.443570e+12
4,2.01611E+11,6,demolitions,11/28/2016,342,1,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",1.445480e+11


In [137]:
df.isnull().sum()

Permit Number                                 0
Permit Type                                   0
Permit Type Definition                        0
Permit Creation Date                          0
Block                                         0
Lot                                           0
Street Number                                 0
Street Number Suffix                      15204
Street Name                                   0
Street Suffix                               194
Unit                                      13217
Unit Suffix                               15185
Description                                  25
Current Status                                0
Current Status Date                           0
Filed Date                                    0
Issued Date                                 635
Completed Date                             6235
First Construction Document Date            639
Structural Notification                   14896
Number of Existing Stories              

For fire only permit empty comlumn value can only imply "NO"

In [138]:
df['Fire Only Permit'] = df['Fire Only Permit'].fillna("N")

# **Drop column with missing values greater than 15000**

In [139]:
threshold = 13000

# Find columns with missing value rate higher than threshold
cols_to_drop = df.columns[df.isnull().sum() > threshold]
df2 = df.drop(columns=cols_to_drop)

In [140]:
df2.isnull().sum()

Permit Number                                0
Permit Type                                  0
Permit Type Definition                       0
Permit Creation Date                         0
Block                                        0
Lot                                          0
Street Number                                0
Street Name                                  0
Street Suffix                              194
Description                                 25
Current Status                               0
Current Status Date                          0
Filed Date                                   0
Issued Date                                635
Completed Date                            6235
First Construction Document Date           639
Number of Existing Stories                3189
Number of Proposed Stories                3386
Fire Only Permit                             0
Permit Expiration Date                    3460
Estimated Cost                            2887
Revised Cost 

# **Fill street suffix with previous values**

In [141]:
df2['Street Suffix'] = df2['Street Suffix'].fillna(method='pad')

In [142]:
df2.isnull().sum()

Permit Number                                0
Permit Type                                  0
Permit Type Definition                       0
Permit Creation Date                         0
Block                                        0
Lot                                          0
Street Number                                0
Street Name                                  0
Street Suffix                                0
Description                                 25
Current Status                               0
Current Status Date                          0
Filed Date                                   0
Issued Date                                635
Completed Date                            6235
First Construction Document Date           639
Number of Existing Stories                3189
Number of Proposed Stories                3386
Fire Only Permit                             0
Permit Expiration Date                    3460
Estimated Cost                            2887
Revised Cost 

In [143]:
df2['Description'] = df2['Description'].fillna('No description available for this product')

# **Fill the revised cost with the Mean, train this with linear regression model**

In [144]:
df2["Revised Cost"].fillna( df2["Revised Cost"].mean(), inplace=True)

In [145]:
df2["Estimated Cost"].fillna( df2["Estimated Cost"].mean(), inplace=True)

In [146]:
df2.shape

(15364, 36)

# **Fill location, zipcode, Neighborhoods - Analysis Boundaries , Supervisor District  with the values after them**

In [147]:
columns_to_fill = ["Location", "Zipcode", "Neighborhoods - Analysis Boundaries", "Supervisor District"]

for column in columns_to_fill:
    df2[column] = df2[column].bfill()

# **Fill contruction type and description with values**

In [148]:
fill_values = {
    'Existing Construction Type': 0 ,
    'Existing Construction Type Description': 'No Description for this construction type',
    'Proposed Construction Type': 0,
    'Proposed Construction Type Description': 'No Description for this construction type',
}

df2.fillna(fill_values, inplace=True)

In [149]:
df2.isnull().sum()

Permit Number                                0
Permit Type                                  0
Permit Type Definition                       0
Permit Creation Date                         0
Block                                        0
Lot                                          0
Street Number                                0
Street Name                                  0
Street Suffix                                0
Description                                  0
Current Status                               0
Current Status Date                          0
Filed Date                                   0
Issued Date                                635
Completed Date                            6235
First Construction Document Date           639
Number of Existing Stories                3189
Number of Proposed Stories                3386
Fire Only Permit                             0
Permit Expiration Date                    3460
Estimated Cost                               0
Revised Cost 

# **Fill missing Completion Date with the highest date and missing Issue Date with the lowest date**


In [150]:
# Fill missing Completion Date with the highest date
df2['Completed Date'] = pd.to_datetime(df2['Completed Date'], errors='coerce')  # Convert to datetime
max_completion_date = df2['Completed Date'].max()  # Find the highest date
df2['Completed Date'].fillna(max_completion_date, inplace=True)  # Fill missing values

# Fill missing Issue Date with the lowest date
df2['Issued Date'] = pd.to_datetime(df2['Issued Date'], errors='coerce')  # Convert to datetime
min_issue_date = df2['Issued Date'].min()  # Find the lowest date
df2['Issued Date'].fillna(min_issue_date, inplace=True)

Fill missing

In [151]:
# Fill missing Completion Date with the highest date
df2['Permit Expiration Date'] = pd.to_datetime(df2['Permit Expiration Date'], errors='coerce')  # Convert to datetime
max_completion_date = df2['Permit Expiration Date'].max()  # Find the highest date
df2['Permit Expiration Date'].fillna(max_completion_date, inplace=True)  # Fill missing values

In [152]:

# Fill missing Completion Date with the highest date
df2['First Construction Document Date'] = pd.to_datetime(df2['First Construction Document Date'], errors='coerce')  # Convert to datetime
max_completion_date = df2['First Construction Document Date'].max()  # Find the highest date
df2['First Construction Document Date'].fillna(max_completion_date, inplace=True)  # Fill missing values

In [153]:
# Calculate the mean of the "Number of Existing Stories" column as an integer
mean_existing_stories = int(df['Number of Existing Stories'].mean())

# Fill missing values with the mean as an integer
df2['Number of Existing Stories'].fillna(mean_existing_stories, inplace=True)

In [154]:
df2['Number of Proposed Stories'].fillna(0, inplace=True)

In [155]:
df2['Proposed Units'].fillna(0, inplace=True)

In [156]:
df2['Existing Units'].fillna(0, inplace=True)

In [157]:
df2['Plansets'].fillna(0, inplace=True)

In [158]:

df2['Existing Use'].fillna("No Existing Use yet", inplace=True)

In [159]:
df2['Proposed Use'].fillna("No Proposed Use yet", inplace=True)

In [160]:
df2.isnull().sum()

Permit Number                             0
Permit Type                               0
Permit Type Definition                    0
Permit Creation Date                      0
Block                                     0
Lot                                       0
Street Number                             0
Street Name                               0
Street Suffix                             0
Description                               0
Current Status                            0
Current Status Date                       0
Filed Date                                0
Issued Date                               0
Completed Date                            0
First Construction Document Date          0
Number of Existing Stories                0
Number of Proposed Stories                0
Fire Only Permit                          0
Permit Expiration Date                    0
Estimated Cost                            0
Revised Cost                              0
Existing Use                    

# New Features

In [161]:
df3 = df2.copy()
df3.shape

(15364, 36)

In [162]:
df3.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number', 'Street Name',
       'Street Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Zipcode', 'Location',
       'Record ID'],
      dtype='object')

The total unit existing after the completion of the proposed unit is given by;

In [163]:
df3['total_unit_after_completion'] = df2['Existing Units']  + df3['Proposed Units']
df3['total_unit_after_completion']

0        143.0
1          0.0
2         78.0
3          2.0
4          0.0
         ...  
15359      2.0
15360      4.0
15361      4.0
15362     16.0
15363      0.0
Name: total_unit_after_completion, Length: 15364, dtype: float64

the lifespan of each permit is given by;

In [164]:
# Convert both columns to datetime if they are not already
df3['Permit Expiration Date'] = pd.to_datetime(df3['Permit Expiration Date'])
df3['Permit Creation Date'] = pd.to_datetime(df3['Permit Creation Date'])

# Subtract the dates to get the duration as a Timedelta
df3['Permit_lifespan'] = df3['Permit Expiration Date'] - df3['Permit Creation Date']

# To get the number of days as integers
df3['Permit_lifespan_days'] = df3['Permit_lifespan'].dt.days


time of completion calculated as the difference between the completion date and the issue date

In [165]:
# Convert both columns to datetime if they are not already
df3['Completed Date'] = pd.to_datetime(df3['Completed Date'])
df3['Issued Date'] = pd.to_datetime(df3['Issued Date'])

# Subtract the dates to get the duration as a Timedelta
df3['time_of_completion'] = df3['Completed Date'] - df3['Issued Date']

# To get the number of days as integers
df3['time_of_completion_days'] = df3['time_of_completion'].dt.days
df3['time_of_completion_days']

0         820
1         187
2        1861
3           6
4          67
         ... 
15359    1701
15360      73
15361      73
15362      55
15363    1701
Name: time_of_completion_days, Length: 15364, dtype: int64

time taken for permit to be issued after it has been filed for

In [166]:
# Convert both columns to datetime if they are not already
df3['Filed Date'] = pd.to_datetime(df3['Filed Date'])
df3['Issued Date'] = pd.to_datetime(df3['Issued Date'])

# Subtract the dates to get the duration as a Timedelta
df3['time_of_issue'] =  df3['Issued Date'] - df3['Filed Date']

# To get the number of days as integers
df3.loc[df3['time_of_issue'].dt.days < 0, 'time_of_issue_days'] = 0

df3.loc[df3['time_of_issue'].dt.days >= 0, 'time_of_issue_days'] = df3['time_of_issue'].dt.days
df3['time_of_issue_days']
df3

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Zipcode,Location,Record ID,total_unit_after_completion,Permit_lifespan,Permit_lifespan_days,time_of_completion,time_of_completion_days,time_of_issue,time_of_issue_days
0,2.01505E+11,4,sign - erect,2015-05-06,326,23,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",...,94102.0,"(37.785719256680785, -122.40852313194863)",1.380610e+12,143.0,547 days,547,820 days,820,187 days,187.0
1,2.01604E+11,4,sign - erect,2016-04-19,306,7,440,Geary,St,remove (e) awning and associated signs.,...,94102.0,"(37.78733980600732, -122.41063199757738)",1.420160e+12,0.0,593 days,593,187 days,187,471 days,471.0
2,2.01605E+11,3,additions alterations or repairs,2016-05-27,595,203,1647,Pacific,Av,installation of separating wall,...,94109.0,"(37.7946573324287, -122.42232562979227)",1.424860e+12,78.0,1995 days,1995,1861 days,1861,-1241 days,0.0
3,2.01611E+11,8,otc alterations permit,2016-11-07,156,11,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,...,94109.0,"(37.79595867909168, -122.41557405519474)",1.443570e+12,2.0,613 days,613,6 days,6,253 days,253.0
4,2.01611E+11,6,demolitions,2016-11-28,342,1,950,Market,St,demolish retail/office/commercial 3-story buil...,...,94102.0,"(37.78315261897309, -122.40950883997789)",1.445480e+11,0.0,733 days,733,67 days,67,368 days,368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15359,2.01306E+11,8,otc alterations permit,2013-06-11,7104,018B,162,Farallones,St,"install finishing plumbing, finish electrical,...",...,94112.0,"(37.7143177382725, -122.45793648914099)",1.307690e+12,2.0,360 days,360,1701 days,1701,0 days,0.0
15360,2.01306E+11,8,otc alterations permit,2013-06-11,3609,81,3549,20th,St,revision to 201208288466 to add a toilet in th...,...,94110.0,"(37.7583635958755, -122.42012861020109)",1.307690e+12,4.0,367 days,367,73 days,73,7 days,7.0
15361,2.01306E+11,8,otc alterations permit,2013-06-11,3609,81,3551,20th,St,revision to 201208288466 to add a toilet in th...,...,94110.0,"(37.7583635958755, -122.42012861020109)",1.307690e+12,4.0,367 days,367,73 days,73,7 days,7.0
15362,2.01306E+11,8,otc alterations permit,2013-06-11,4260,13,2334,25th,St,"interior renovation unit #2334, new kitchen fi...",...,94107.0,"(37.752135529825054, -122.40206132582264)",1.307690e+12,16.0,360 days,360,55 days,55,0 days,0.0


check if there is any change and if the change is either higher or lower and add a column to indicate the type of chage that happened

In [167]:
cond = [
    (df3['Estimated Cost'] - df3['Revised Cost']) > 0,
    (df3['Estimated Cost'] - df3['Revised Cost']) == 0,
    (df3['Estimated Cost'] - df3['Revised Cost']) < 0,
]

choices = ['decreased', 'No change', 'increased']
df3['cost_changed'] = np.select(cond, choices, default='unknown')

In [168]:
df3['Current Status'].value_counts()

complete      9103
issued        5258
filed          444
expired        227
cancelled      146
withdrawn      105
reinstated      34
approved        32
suspend         10
revoked          3
appeal           1
plancheck        1
Name: Current Status, dtype: int64

In [169]:
df3['percentage_story_increase'] = df3['Number of Existing Stories']/ ( df3['Number of Existing Stories'] + df3['Number of Proposed Stories'])
df3['percentage_story_increase']

0        1.0
1        1.0
2        0.5
3        0.5
4        1.0
        ... 
15359    0.5
15360    0.5
15361    0.5
15362    0.5
15363    1.0
Name: percentage_story_increase, Length: 15364, dtype: float64

In [170]:
df3.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number', 'Street Name',
       'Street Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Zipcode', 'Location',
       'Record ID', 'total_unit_after_completion', 'Permit_lifespan',
       'Permit_lifespan_days', 'time_of_completion', 'time_of_completion_days',
       'time_of_issue', 'time_o

## Encoding of Categorical variables

In [175]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder

encoder = LabelEncoder()
categorical_columns = ['Permit Type Definition','Street Suffix','Existing Construction Type Description','Proposed Construction Type Description','Fire Only Permit','Lot','Description', 'Current Status', 'cost_changed']
df3_4 = df3[categorical_columns].apply(encoder.fit_transform)

Percentage increase in number of stories

In [183]:
df3_4.head()

,Permit Type Definition,Street Suffix,Existing Construction Type Description,Proposed Construction Type Description,Fire Only Permit,Lot,Description,Current Status,cost_changed,total_unit_after_completion,Permit_lifespan_days,time_of_completion_days,time_of_issue_days,percentage_story_increase
0,0.857143,12,3,0,0,339,3301,4,0,143.0,547,820,187.0,1.0
1,0.857143,12,3,0,0,473,6095,6,2,0.0,593,187,471.0,1.0
2,0.000000,1,1,1,0,318,4009,11,2,78.0,1995,1861,0.0,0.5
3,0.714286,1,5,5,0,241,6659,3,0,2.0,613,6,253.0,0.5
4,0.142857,12,3,0,0,229,2643,6,0,0.0,733,67,368.0,1.0


combine the new features with this

In [177]:
df3_4['total_unit_after_completion'] = df3['total_unit_after_completion']
df3_4['Permit_lifespan_days'] = df3['Permit_lifespan_days']
df3_4['time_of_completion_days'] = df3['time_of_completion_days']
df3_4['time_of_issue_days'] = df3['time_of_issue_days']
df3_4['percentage_story_increase'] = df3['percentage_story_increase']

In [179]:
df3_4.columns

Index(['Permit Type Definition', 'Street Suffix',
       'Existing Construction Type Description',
       'Proposed Construction Type Description', 'Fire Only Permit', 'Lot',
       'Description', 'Current Status', 'cost_changed',
       'total_unit_after_completion', 'Permit_lifespan_days',
       'time_of_completion_days', 'time_of_issue_days',
       'percentage_story_increase'],
      dtype='object')

## Normalize the df

## export to csv file

In [185]:
df3_4.to_csv('transformed_data.csv', index=False)